In [8]:
import sys
import os

# Add the src/models folder to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'models')))

# Now you can import the Model class
from mlp import Model as MLP